In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; font-size:100%;}</style>"))

In [ ]:
from pprint import pprint
from collections import Counter
import csv
import datetime
import json

from bson.son import SON
from bson import json_util

In [ ]:
!pip install pyspark

In [ ]:
!pip install pyarrow

In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
session_name = "USA"

spark = SparkSession.builder.appName(session_name).getOrCreate()
spark

In [ ]:
sc = spark.sparkContext

In [ ]:
usa_list = [
  ('AL', 'Alabama', 'State', '7', 'South', 'East South Central', 'Montgomery', '32.371002', '-86.394550', '4863300'),
  ('AK', 'Alaska', 'State', '1', 'West', 'Pacific', 'Juneau', '58.317506', '-134.417574', '741894'),
  ('AZ', 'Arizona', 'State', '9', 'West', 'Mountain', 'Phoenix', '33.445968', '-112.101734', '6931071'),
  ('AR', 'Arkansas', 'State', '4', 'South', 'West South Central', 'Little Rock', '34.745992', '-92.292611', '2988248'),
  ('CA', 'California', 'State', '53', 'West', 'Pacific', 'Sacramento', '38.579504', '-121.487203', '39250017'),
  ('CO', 'Colorado', 'State', '7', 'West', 'Mountain', 'Denver', '39.737859', '-104.991760', '5540545'),
  ('CT', 'Connecticut', 'State', '5', 'Northeast', 'New England', 'Hartford', '41.763762', '-72.683864', '3576452'),
  ('DC', 'Washington, D.C.', 'District', None, 'South', 'South Atlantic', 'District of Columbia', '38.906007', '-77.037571', '681170'),
  ('DE', 'Delaware', 'State', '1', 'South', 'South Atlantic', 'Dover', '39.157875', '-75.525093', '952065'),
  ('FL', 'Florida', 'State', '27', 'South', 'South Atlantic', 'Tallahassee', '30.438709', '-84.277800', '20612439'),
  ('GA', 'Georgia', 'State', '14', 'South', 'South Atlantic', 'Atlanta', '33.749711', '-84.386617', '10310371'),
  ('HI', 'Hawaii', 'State', '2', 'West', 'Pacific', 'Honolulu', '21.307870', '-157.860530', '1428557'),
  ('ID', 'Idaho', 'State', '2', 'West', 'Mountain', 'Boise', '43.618434', '-116.211584', '1683140'),
  ('IL', 'Illinois', 'State', '18', 'Midwest', 'East North Central', 'Springfield', '39.782024', '-89.644054', '12801539'),
  ('IN', 'Indiana', 'State', '9', 'Midwest', 'East North Central', 'Indianapolis', '39.764395', '-86.149923', '6633053'),
  ('IA', 'Iowa', 'State', '4', 'Midwest', 'West North Central', 'Des Moines', '41.599520', '-93.601024', '3134693'),
  ('KS', 'Kansas', 'State', '4', 'Midwest', 'West North Central', 'Topeka', '39.057036', '-95.688120', '2907289'),
  ('KY', 'Kentucky', 'State', '6', 'South', 'East South Central', 'Frankfort', '38.200376', '-84.872634', '4436974'),
  ('LA', 'Louisiana', 'State', '6', 'South', 'West South Central', 'Baton Rouge', '30.458030', '-91.133393', '4681666'),
  ('ME', 'Maine', 'State', '2', 'Northeast', 'New England', 'Augusta', '44.310730', '-69.778454', '1331479'),
  ('MD', 'Maryland', 'State', '8', 'South', 'South Atlantic', 'Annapolis', '38.978390', '-76.492150', '6016447'),
  ('MA', 'Massachusetts', 'State', '9', 'Northeast', 'New England', 'Boston', '42.360302', '-71.055442', '6811779'),
  ('MI', 'Michigan', 'State', '14', 'Midwest', 'East North Central', 'Lansing', '42.733126', '-84.554570', '9928300'),
  ('MN', 'Minnesota', 'State', '8', 'Midwest', 'West North Central', 'St. Paul', '44.953796', '-93.087686', '5519952'),
  ('MS', 'Mississippi', 'State', '4', 'South', 'East South Central', 'Jackson', '32.298702', '-90.177903', '2988726'),
  ('MO', 'Missouri', 'State', '8', 'Midwest', 'West North Central', 'Jefferson City', '38.576513', '-92.169960', '6093000'),
  ('MT', 'Montana', 'State', '1', 'West', 'Mountain', 'Helena', '46.587808', '-112.024277', '1042520'),
  ('NE', 'Nebraska', 'State', '3', 'Midwest', 'West North Central', 'Lincoln', '40.824494', '-96.680087', '1907116'),
  ('NV', 'Nevada', 'State', '4', 'West', 'Mountain', 'Carson City', '39.162426', '-119.763287', '2940058'),
  ('NH', 'New Hampshire', 'State', '2', 'Northeast', 'New England', 'Concord', '43.208013', '-71.539261', '1334795'),
  ('NJ', 'New Jersey', 'State', '12', 'Northeast', 'Middle Atlantic', 'Trenton', '40.217190', '-74.741850', '8944469'),
  ('NM', 'New Mexico', 'State', '3', 'West', 'Mountain', 'Santa Fe', '35.687214', '-105.937719', '2081015'),
  ('NY', 'New York', 'State', '27', 'Northeast', 'Middle Atlantic', 'Albany', '42.652299', '-73.754720', '19745289'),
  ('NC', 'North Carolina', 'State', '13', 'South', 'South Atlantic', 'Raleigh', '35.779712', '-78.632011', '10146788'),
  ('ND', 'North Dakota', 'State', '1', 'Midwest', 'West North Central', 'Bismarck', '46.807880', '-100.782162', '757952'),
  ('OH', 'Ohio', 'State', '16', 'Midwest', 'East North Central', 'Columbus', '39.961432', '-82.998850', '11614373'),
  ('OK', 'Oklahoma', 'State', '5', 'South', 'West South Central', 'Oklahoma City', '35.468588', '-97.496371', '3923561'),
  ('OR', 'Oregon', 'State', '5', 'West', 'Pacific', 'Salem', '44.943177', '-123.034389', '4093465'),
  ('PA', 'Pennsylvania', 'State', '18', 'Northeast', 'Middle Atlantic', 'Harrisburg', '40.272972', '-76.883172', '12784227'),
  ('RI', 'Rhode Island', 'State', '2', 'Northeast', 'New England', 'Providence', '41.823384', '-71.410697', '1056426'),
  ('SC', 'South Carolina', 'State', '7', 'South', 'South Atlantic', 'Columbia', '34.000421', '-81.028292', '4961119'),
  ('SD', 'South Dakota', 'State', '1', 'Midwest', 'West North Central', 'Pierre', '44.366713', '-100.342667', '865454'),
  ('TN', 'Tennessee', 'State', '9', 'South', 'East South Central', 'Nashville', '36.164388', '-86.778386', '6651194'),
  ('TX', 'Texas', 'State', '36', 'South', 'West South Central', 'Austin', '30.267368', '-97.737616', '27862596'),
  ('UT', 'Utah', 'State', '4', 'West', 'Mountain', 'Salt Lake City', '40.759686', '-111.891648', '3051217'),
  ('VT', 'Vermont', 'State', '1', 'Northeast', 'New England', 'Montpelier', '44.260050', '-72.573870', '624594'),
  ('VA', 'Virginia', 'State', '11', 'South', 'South Atlantic', 'Richmond', '37.540431', '-77.432838', '8411808'),
  ('WA', 'Washington', 'State', '10', 'West', 'Pacific', 'Olympia', '47.038158', '-122.897302', '7288000'),
  ('WV', 'West Virginia', 'State', '3', 'South', 'South Atlantic', 'Charleston', '38.349233', '-81.632901', '1831102'),
  ('WI', 'Wisconsin', 'State', '8', 'Midwest', 'East North Central', 'Madison', '43.072434', '-89.399899', '5778708'),
  ('WY', 'Wyoming', 'State', '1', 'West', 'Mountain', 'Cheyenne', '41.140029', '-104.821135', '585501'),
  ('PR', 'Puerto Rico', 'Territory', None, 'South', 'South Atlantic', 'San Juan', '18.465189', '-66.101552', '3411307'),
  ('GU', 'Guam', 'Territory', None, 'South', 'Pacific', 'Hagåtña', '13.474292', '144.753015', '162742'),
  ('VI', 'U.S. Virgin Islands', 'Territory', None, 'South', 'South Atlantic', 'Charlotte Amalie', '18.338427', '-64.938825', '106792'),
  ('AS', 'American Samoa', 'Territory', None, 'South', 'Pacific', 'Pago Pago', '-14.271918', '-170.700754', '54194'),
  ('MP', 'Northern Mariana Islands', 'Territory', None, 'South', 'Pacific', 'Saipan', '15.154782', '145.730151', '53467'),
]

In [ ]:
usa_columns = (
  'state_abbr',
  'state_name',
  'state_type',
  'state_seats',
  'state_region',
  'state_division',
  'state_capital',
  'capital_latitude',
  'capital_longitude',
  'state_population',
)

In [ ]:
def numint(x):
    try:
        return int(x)
    except:
        return 0
      
for item in ['', '55', 500, 'five', '5_000_000', None]:
    print(numint(item))

In [ ]:
usa_dict = []

for tup in usa_list:
    row = dict(zip(usa_columns, tup))
    row['state_seats'] = numint(row['state_seats'])
    row['state_population'] = numint(row['state_population'])
    row['capital_latitude'] = float(row['capital_latitude'])
    row['capital_longitude'] = float(row['capital_longitude'])  
    row['append_date_time'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    usa_dict.append(row)

In [ ]:
usa_dict

In [ ]:
Counter(row['state_region'] for row in usa_dict)

In [ ]:
Counter((row['state_region'], row['state_division']) for row in usa_dict)

In [ ]:
# save as CSV (Comma Separated Values)
with open('usa_dataset.csv', mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=usa_dict[0].keys())
    writer.writeheader()
    writer.writerows(usa_dict)

In [ ]:
# save as JSON (JavaScript Object Notation)
with open(file='usa_dataset.json', mode='w') as fp:
    json.dump(fp=fp, obj=usa_dict, indent=2, default=json_util.default)

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime

In [ ]:
# This creates a database (if it does not exist) or opens the existing one
engine = create_engine('sqlite:///usa_data.db', echo=False)

In [ ]:
meta = MetaData()

usa = Table(
    'usa_table', meta,
    Column('state_abbr', String),
    Column('state_name', String),
    Column('state_type', String),
    Column('state_seats', Integer),
    Column('state_region', String),
    Column('state_division', String),
    Column('state_capital', String),
    Column('capital_latitude', Float),
    Column('capital_longitude', Float),
    Column('state_population', Integer),
    Column('append_date_time', String),
)

meta.create_all(engine)

conn = engine.connect()

In [ ]:
# SQL Alchemy is able to interpret this format for inserting records to the table

conn.execute(usa.insert(), usa_dict)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('usa_dataset.csv', parse_dates=['append_date_time'])

df.info()

In [ ]:
df.head()

In [ ]:
df = pd.read_json('usa_dataset.json', convert_dates=True)
df.info()

In [ ]:
df.head()

In [ ]:
df = pd.read_sql(sql='select * from usa_table', con=conn, parse_dates=['append_date_time'])
df.info()

In [ ]:
fp = 'usa_dataset.csv'
df = spark.read.format("csv").option('header','true').option('inferSchema','true').load(fp)
df.printSchema()

In [ ]:
df = df.withColumn('append_date_time', to_timestamp(col('append_date_time')))
df.printSchema()

In [ ]:
df.show(5)

In [ ]:
df.filter(df['state_seats']==0).show()